In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch

In [2]:
#create custom data class

class CustomDataset(Dataset):
    def __init__(self,low_idx,high_idx):
        self.data = pd.read_csv("./dataset/IRIS.csv", sep=",", skiprows=range(low_idx,high_idx), header=1)
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sepal_length = torch.tensor(self.data.iloc[idx, 0])
        sepal_width = self.data.iloc[idx, 1]
        petal_length = self.data.iloc[idx, 2]
        petal_width = self.data.iloc[idx, 3] 
        label = self.data.iloc[idx, 4]
        
        tensor_data = torch.tensor([sepal_length,sepal_width,sepal_width,sepal_width],dtype=torch.float32)
        
        return tensor_data , label

In [3]:
#just my afwull way to seperate training and testing dataset
# 80% training , 20% Test

train_dataset = CustomDataset(120,149)
test_dataset = CustomDataset(1,120)


In [4]:
#create train_dataloader and test_dataloader
# 80% for training, 20% of data for testing -> 149*0.8 = 119.2 -> 120 , 29
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True) 
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)


In [5]:
#4 inputs and 3  outputs 
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_stack =  nn.Sequential(
         nn.Linear(4,128),
         nn.ReLU(),
         nn.Linear(128,64),
         nn.ReLU(),
         nn.Linear(64,3),
         )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits

In [6]:
#map labels into tensor of shape [1, 0, 0]
def transform_label(label_data):
    data = []
    for i in label_data:
        if i == "Iris-setosa":
            data.append(torch.tensor([1,0,0], dtype=torch.float32))
        elif i == "Iris-versicolor": 
            data.append(torch.tensor([0,1,0], dtype=torch.float32))
        else:
            data.append(torch.tensor([0,0,1], dtype=torch.float32))
    return torch.stack(data)

In [7]:
#train_labels, X_train = next(iter(train_loader))
X_train, train_labels = next(iter(train_loader))
train_labels = transform_label(train_labels)
X_train.shape, train_labels.shape, X_train.dtype, train_labels.dtype

(torch.Size([10, 4]), torch.Size([10, 3]), torch.float32, torch.float32)

In [8]:
X_train

tensor([[4.9000, 2.5000, 2.5000, 2.5000],
        [5.4000, 3.4000, 3.4000, 3.4000],
        [5.7000, 2.9000, 2.9000, 2.9000],
        [6.6000, 2.9000, 2.9000, 2.9000],
        [5.7000, 2.8000, 2.8000, 2.8000],
        [5.1000, 3.8000, 3.8000, 3.8000],
        [7.2000, 3.6000, 3.6000, 3.6000],
        [5.1000, 3.8000, 3.8000, 3.8000],
        [6.2000, 2.9000, 2.9000, 2.9000],
        [5.4000, 3.0000, 3.0000, 3.0000]])

In [9]:
train_labels

tensor([[0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.]])

In [10]:
#create model
lr = 0.01
model = NeuralNetwork()
optim = torch.optim.Adam(model.parameters(), lr=lr)
loss = torch.nn.CrossEntropyLoss()
#Weights are by default torch.32 not 64 --> error message 

In [11]:
n_iters = 10
for epochs in range(n_iters):
    #forward pass and loss
    #test_labels, X_test = next(iter(test_loader))
    print(f"\n epochs: {epochs+1}/{n_iters} \n")
    for i,(inputs, labels) in enumerate(train_loader):
        out = model(inputs)
        train_labels = transform_label(labels)
        l = loss(out, train_labels)
        #backward pass
        l.backward()
        #update weights
        optim.step()
        optim.zero_grad()
    
        if i % 1 == 0:
            print(f" steps: {i+1}, loss : {l.item()}")


 epochs: 1/10 

 steps: 1, loss : 1.2101413011550903
 steps: 2, loss : 1.2012776136398315
 steps: 3, loss : 1.1544098854064941
 steps: 4, loss : 1.1238055229187012
 steps: 5, loss : 1.0931551456451416
 steps: 6, loss : 0.8755502700805664
 steps: 7, loss : 0.8547183871269226
 steps: 8, loss : 0.7368665933609009
 steps: 9, loss : 1.287074327468872
 steps: 10, loss : 1.5251169204711914
 steps: 11, loss : 0.6588473320007324
 steps: 12, loss : 0.8195764422416687

 epochs: 2/10 

 steps: 1, loss : 0.6243541240692139
 steps: 2, loss : 0.5123398900032043
 steps: 3, loss : 0.4553864598274231
 steps: 4, loss : 0.49558600783348083
 steps: 5, loss : 0.8610669374465942
 steps: 6, loss : 0.35583755373954773
 steps: 7, loss : 1.0581166744232178
 steps: 8, loss : 0.721039891242981
 steps: 9, loss : 0.751634955406189
 steps: 10, loss : 0.5298471450805664
 steps: 11, loss : 0.6286306977272034
 steps: 12, loss : 0.6789177656173706

 epochs: 3/10 

 steps: 1, loss : 0.47378215193748474
 steps: 2, loss : 

In [12]:
input_test = torch.tensor([4.9,3.1,1.5,0.1])

In [13]:
model(input_test)

tensor([-3.7291,  1.9326,  1.3174], grad_fn=<AddBackward0>)

In [14]:
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)
X_test, test_labels = next(iter(train_loader))
test_labels = transform_label(test_labels)

In [15]:
x_label_predicted = model(X_test)

In [16]:
x_label_predicted

tensor([[-3.0981,  1.6812,  1.0963],
        [-4.1906,  2.2157,  1.5601],
        [-3.3776,  1.8180,  1.2103],
        [ 4.3282, -1.1395, -2.4044],
        [-3.1913,  1.7268,  1.1343],
        [-4.2353,  2.2379,  1.5759],
        [-4.1975,  2.2177,  1.5731],
        [ 3.8486, -0.9979, -2.1862],
        [-1.8436,  1.0771,  0.5752],
        [ 2.5256, -0.5872, -1.5666]], grad_fn=<AddmmBackward0>)

In [17]:
_, x_label_predicted_hat = torch.max(x_label_predicted,1)

In [18]:
x_label_predicted_hat

tensor([1, 1, 1, 0, 1, 1, 1, 0, 1, 0])

In [19]:
def final_x(x_hat):
    list_tensors = []
    for i in x_hat:
        final_tensor = torch.zeros(1,3)
        final_tensor[0:,i] = 1
        list_tensors.append(final_tensor)
    return torch.cat(list_tensors)
    
    # tensor[0:,1] = 4 to accses specific value 

In [20]:
final_x = final_x(x_label_predicted_hat)

In [21]:
#final_x, test_labels
#print(f"{})

In [22]:
def get_accuracy(model_prediction, test_labels):
    idx = 0
    number_pred = 0
    counter = 0
    while idx < 10:
        print(f" \n Tensor[{idx+1}]")
        for i in range(3):
            print(f"model: {model_prediction[idx][i].item()}, label: {test_labels[idx][i].item()}")
            if model_prediction[idx][i].item() == test_labels[idx][i].item():

                counter += 1
                if counter == 3:
                    number_pred += 1
        counter = 0
          
        idx +=1
    accuracy = number_pred/10 *100
    return print(f"\naccuracy of model {number_pred/10 *100}%")

In [23]:
get_accuracy(final_x, test_labels)

 
 Tensor[1]
model: 0.0, label: 0.0
model: 1.0, label: 1.0
model: 0.0, label: 0.0
 
 Tensor[2]
model: 0.0, label: 0.0
model: 1.0, label: 1.0
model: 0.0, label: 0.0
 
 Tensor[3]
model: 0.0, label: 0.0
model: 1.0, label: 1.0
model: 0.0, label: 0.0
 
 Tensor[4]
model: 1.0, label: 1.0
model: 0.0, label: 0.0
model: 0.0, label: 0.0
 
 Tensor[5]
model: 0.0, label: 0.0
model: 1.0, label: 1.0
model: 0.0, label: 0.0
 
 Tensor[6]
model: 0.0, label: 0.0
model: 1.0, label: 0.0
model: 0.0, label: 1.0
 
 Tensor[7]
model: 0.0, label: 0.0
model: 1.0, label: 1.0
model: 0.0, label: 0.0
 
 Tensor[8]
model: 1.0, label: 1.0
model: 0.0, label: 0.0
model: 0.0, label: 0.0
 
 Tensor[9]
model: 0.0, label: 0.0
model: 1.0, label: 1.0
model: 0.0, label: 0.0
 
 Tensor[10]
model: 1.0, label: 1.0
model: 0.0, label: 0.0
model: 0.0, label: 0.0

accuracy of model 90.0%


In [25]:
import accuracy_model as ac
test_model_acc = ac.data_loader
test_model_acc.accuracy(model, test_dataset, batch_size=10)

 
 Tensor[1]
model: 0.0, label: 0.0
model: 1.0, label: 0.0
model: 0.0, label: 1.0
 
 Tensor[2]
model: 0.0, label: 0.0
model: 1.0, label: 0.0
model: 0.0, label: 1.0
 
 Tensor[3]
model: 0.0, label: 0.0
model: 1.0, label: 0.0
model: 0.0, label: 1.0
 
 Tensor[4]
model: 0.0, label: 0.0
model: 1.0, label: 0.0
model: 0.0, label: 1.0
 
 Tensor[5]
model: 0.0, label: 0.0
model: 1.0, label: 0.0
model: 0.0, label: 1.0
 
 Tensor[6]
model: 0.0, label: 0.0
model: 1.0, label: 0.0
model: 0.0, label: 1.0
 
 Tensor[7]
model: 0.0, label: 0.0
model: 1.0, label: 0.0
model: 0.0, label: 1.0
 
 Tensor[8]
model: 0.0, label: 0.0
model: 1.0, label: 0.0
model: 0.0, label: 1.0
 
 Tensor[9]
model: 0.0, label: 0.0
model: 1.0, label: 0.0
model: 0.0, label: 1.0
 
 Tensor[10]
model: 0.0, label: 0.0
model: 1.0, label: 0.0
model: 0.0, label: 1.0
 
 Tensor[1]
model: 0.0, label: 0.0
model: 1.0, label: 0.0
model: 0.0, label: 1.0
 
 Tensor[2]
model: 0.0, label: 0.0
model: 1.0, label: 0.0
model: 0.0, label: 1.0
 
 Tensor[3]
mo

UnboundLocalError: local variable 'summe' referenced before assignment

In [ ]:
# save model 
torch.save(model.state_dict(), "./model")